# 사전 준비

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.3 MB/s eta 0:00:00


**경고 메시지 끄기**

In [2]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

**사전학습모델과 토크나이저(KcELECTRA, KoGPT2) 불러오기**

In [3]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, GPT2LMHeadModel, PreTrainedTokenizerFast

ELECTRA_tokenizer = ElectraTokenizer.from_pretrained("beomi/KcELECTRA-base")
CurseELECTRA_model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base")   # 비속어 감지의 사전학습 모델로는 KcELECTRA 사용
ChatELECTRA_model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels = 3)    # 챗봇 데이터 label 분류모델([0, 1, 2])

GPT2_tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',       # 문장 시작토큰
    eos_token='</s>',       # 문장 마지막토큰
    unk_token='<unk>',      # 어휘에 없는 토큰
    pad_token='<pad>',      # 크기 맞추기 토큰
    mask_token='<mask>',     # 마스킹 토큰
)
GPT2_Model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')   # 챗봇의 사전학습 모델로는 KoGPT2 사용

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


...

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.de

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

# 데이터 불러오기

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import urllib.request

In [5]:
curse_data = pd.read_table("/content/drive/Othercomputers/내 컴퓨터/Curse_Detection_Chatbot/Curse-detection-data/dataset.txt", 
                           names=["text", "label"], sep="|", header=None)
curse_data

,text,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5819,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5820,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5821,너가 한 말 중에,0
5822,제갈대중 ㅇㅂ,0


In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
chat_data = pd.read_csv('ChatBotData.csv')

chat_data = chat_data.sample(frac=1).reset_index(drop=True)
chat_data

,Q,A,label
0,기다림 속에 갇혀 있어,그 속에서 나오세요.,2
1,헤어짐 이후,마음이 좀 가라앉았길 바랄게요.,1
2,능력이 없는 거 같아,당신의 능력을 잘 찾아보세요.,0
3,뭐하면 시간이 잘 갈까,멍 때리기,0
4,맛집 가고 싶어,벌써 먹고 싶어요.,0
...,...,...,...
11818,친구랑 말다툼했어,싸우면서 정 들 거예요.,0
11819,복 받겠지?,뿌린대로 거둘 거예요.,0
11820,내일 날씨 어때?,날씨 어플에 물어보세요.,0
11821,내가 그냥 학교 후배래,그런 사람 신경쓰지 마세요.,2


# 텍스트 전처리

**전처리 함수 정의**

In [7]:
!pip install soynlp
!pip install emoji==1.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 KB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=ebe8c63ad4e4c3f277959437c981d0c84ad6e92eae0708285645aea116ccd062
  Stored in directory: /root/.cache/pip/wheels/5e/8c/80/c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


In [8]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [9]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
special_symbol = re.compile(
    r'([.,?!/@$%~％·∼()\x21-\x2F\x3A-\x40\x5B-\x60\x7B-\x7E])\1{1,}')

In [10]:
def clean(x):
    x = pattern.sub(' ', x)                     # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)                  # URL 제거
    x = special_symbol.sub('\\1'*1, x)          # 반복되는 특수문자의 축약 횟수 1개로 줄임
    x = x.strip()                               # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

**전처리 및 데이터 분할**

텍스트 전처리는 비속어 감지 데이터에만 적용한다. (챗봇 데이터와 달리 댓글의 성격을 띄기 떄문에 무의미한 반복의 텍스트가 존재하기 때문)

In [11]:
import pandas as pd

In [12]:
# 비속어 감지 데이터
# train : validation = 3 : 1 (5824 = 2^6 * 7 * 13)

train_curse_text = [clean(curse_data['text'][idx]) for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_text = [clean(curse_data['text'][idx]) for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

train_curse_label = [curse_data['label'][idx] for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_label = [curse_data['label'][idx] for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

In [13]:
# 챗봇 데이터(Q, A, label)
# train : validation = 5 : 2 (11823 = 3 * 7 * 563)

train_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (0, int((chat_data.shape[0]/7)*5)))
val_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (int((chat_data.shape[0]/7)*5), int((chat_data.shape[0]/7)*7)))

In [14]:
# 챗봇 데이터(Q, label)
# train : validation = 5 : 2 (11823 = 3 * 7 * 563)

train_chat_Qtext = [train_chat_data['Q'][idx] for idx in range (0, train_chat_data.shape[0])]
val_chat_Qtext = [val_chat_data['Q'][idx] for idx in range (train_chat_data.shape[0], train_chat_data.shape[0]+val_chat_data.shape[0])]

train_chat_label = [train_chat_data['label'][idx] for idx in range (0, train_chat_data.shape[0])]
val_chat_label = [val_chat_data['label'][idx] for idx in range (train_chat_data.shape[0], train_chat_data.shape[0]+val_chat_data.shape[0])]

# 데이터 구축, 토크나이징

**KoGPT2 스페셜토큰 확인**

In [15]:
for i in range (10):
    print("index : ",i," =  tokens : ",GPT2_tokenizer.decode(i))

index :  0  =  tokens :  <s>
index :  1  =  tokens :  </s>
index :  2  =  tokens :  <usr>
index :  3  =  tokens :  <pad>
index :  4  =  tokens :  <sys>
index :  5  =  tokens :  <unk>
index :  6  =  tokens :  <mask>
index :  7  =  tokens :  <d>
index :  8  =  tokens :  </d>
index :  9  =  tokens :  <unused0>


In [16]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [17]:
# 비속어 감지, 챗봇 데이터 분류 데이터셋

class ClfDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

**비속어 감지 데이터**

In [18]:
# 토크나이징

train_input_token = ELECTRA_tokenizer(train_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")
val_input_token = ELECTRA_tokenizer(val_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")

In [19]:
# 데이터셋

train_curse_dataset = ClfDataset(train_input_token, train_curse_label)
val_curse_dataset = ClfDataset(val_input_token, val_curse_label)

In [20]:
# 데이터로더

train_curse_loader = DataLoader(train_curse_dataset, shuffle=True, batch_size=8)
val_curse_loader = DataLoader(val_curse_dataset, shuffle=True, batch_size=8)

**챗봇 데이터**

In [21]:
# 토크나이징 (챗봇 데이터(Q, label))

train_input_token = ELECTRA_tokenizer(train_chat_Qtext, truncation=True, padding=True, max_length=256, return_tensors="pt")
val_input_token = ELECTRA_tokenizer(val_chat_Qtext, truncation=True, padding=True, max_length=256, return_tensors="pt")

In [22]:
# 데이터셋

train_chat_clf_dataset = ClfDataset(train_input_token, train_chat_label)
val_chat_clf_dataset = ClfDataset(val_input_token, val_chat_label)

In [23]:
# 데이터로더

train_chat_clf_loader = DataLoader(train_chat_clf_dataset, shuffle=True, batch_size=16)
val_chat_clf_loader = DataLoader(val_chat_clf_dataset, shuffle=True, batch_size=16)

In [24]:
# 챗봇(Q, A) 데이터셋 (토크나이징 포함)
# <s> <usr> q <unused0> sent <sys> a </s>

class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=64):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"
        self.a_token = "<sys>"
        self.sent_token = "<unused0>"       # 미사용 토큰 하나를 sent 토큰으로 사용
        self.bos = GPT2_tokenizer.bos_token
        self.eos = GPT2_tokenizer.eos_token
        self.mask = GPT2_tokenizer.mask_token
        self.tokenizer = GPT2_tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        index = self._data.iloc[idx]

        q = index["Q"]  # 질문
        a = index["A"]  # 답변
        sent = str(index["label"])   # label

        q_toked = self.tokenizer.tokenize(self.bos + self.q_token + q + self.sent_token + sent)      # 질문 + label
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)      # 답 
        a_len = len(a_toked)

        # 질문의 길이가 최대길이보다 클때
        if q_len > self.max_len: 
            q_toked = q_toked[-(int(self.max_len / 2)):]   # 질문길이를 최대길이의 반으로 줄임
            q_len = len(q_toked)

        # 질문 + 답변 길이가 최대길이보다 클때
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        # 답변의 길이 = 최대길이 - 질문길이

            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              # 답변의 길이를 최대길이 - 질문길이
                
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 질문 + 답변 토큰을 index로 변환   
        token = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 padding
        while len(token) < self.max_len:
            token += [self.tokenizer.pad_token_id]

        # attention(어텐션마스크) = 질문+답변 길이 1 + 나머지(패딩) 0
        attention = [1]*(q_len+a_len) + [0]*(self.max_len - q_len - a_len)

        # token_type_ids(세그먼트 정보) = 질문 0 + 답변 1 + 나머지 0
        token_type = [0]*q_len + [1]*a_len + [0]*(self.max_len - q_len - a_len)

        # label = (질문+sent)*<mask> + 답변 + 패딩
        label = [self.mask]*(q_len) + a_toked[0:]
        # label을 index로 변환
        label = self.tokenizer.convert_tokens_to_ids(label)
        # 최대길이만큼 padding
        while len(label) < self.max_len:
            label += [self.tokenizer.pad_token_id]

        
        # 질문 + 답변, 어텐션마스크, 세그먼트 정보, 답변
        return (token, attention, token_type, label)

In [25]:
# 데이터셋

train_chat_dataset = ChatbotDataset(train_chat_data, max_len=64)
val_chat_dataset = ChatbotDataset(val_chat_data, max_len=64)

In [26]:
# collate_fn 구성

def collate_batch(batch):
    token_ids = [item[:][0] for item in batch]
    attention_mask = [item[:][1] for item in batch]
    token_tpye_ids = [item[:][2] for item in batch]
    label_ids = [item[:][3] for item in batch]

    return torch.LongTensor(token_ids), torch.LongTensor(attention_mask), torch.LongTensor(token_tpye_ids), torch.LongTensor(label_ids)

In [27]:
# 데이터로더

train_chat_loader = DataLoader(train_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)
val_chat_loader = DataLoader(val_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)

# 모델 학습

**모델 파라미터 설정**

In [28]:
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm as tqdm_auto
from sklearn.metrics import accuracy_score

In [29]:
# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
# KcELECTRA(비속어 감지 모델) 파라미터 설정

CurseELECTRA_epochs = 5
CurseELECTRA_learning_rate = 5e-5

CurseELECTRA_optimizer = torch.optim.AdamW(CurseELECTRA_model.parameters(), lr=CurseELECTRA_learning_rate)
CurseELECTRA_criterion = torch.nn.CrossEntropyLoss()

CurseELECTRA_step = 0
CurseELECTRA_eval_steps = len(train_curse_loader)        # 훈련 배치수

In [31]:
# KcELECTRA(챗봇 데이터 label 분류모델) 파라미터 설정

ChatELECTRA_epochs = 5
ChatELECTRA_learning_rate = 5e-5

ChatELECTRA_optimizer = torch.optim.AdamW(ChatELECTRA_model.parameters(), lr=ChatELECTRA_learning_rate)
ChatELECTRA_criterion = torch.nn.BCEWithLogitsLoss()

ChatELECTRA_step = 0
ChatELECTRA_eval_steps = len(train_chat_clf_loader)        # 훈련 배치수

In [32]:
# KoGPT2 파라미터 설정

GPT2_epoch = 10
GPT2_learning_rate = 1e-4

GPT2_optimizer = torch.optim.AdamW(GPT2_Model.parameters(), lr=GPT2_learning_rate)

GPT2_step = 0
GPT2_eval_steps = len(train_chat_loader)        # 훈련 배치수

In [33]:
CurseELECTRA_model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [34]:
ChatELECTRA_model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [35]:
GPT2_Model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

**KcELECTRA(비속어 감지 모델) 학습 진행**

In [36]:
for epoch in range(CurseELECTRA_epochs):
    n = 0
    train_accuracy = 0
    loss = 0
    train_loss = 0.0
    
    CurseELECTRA_model.train()
    for batch in tqdm_auto(train_curse_loader, mininterval=0.01, leave=True):
        CurseELECTRA_optimizer.zero_grad()     # 그래디언트 초기화

        # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = CurseELECTRA_model(**inputs).logits    # 모형으로 결과 예측

        loss = CurseELECTRA_criterion(outputs, labels)
        train_loss += loss
        
        loss.backward()
        CurseELECTRA_optimizer.step()

        # eval_steps 마다 loss를 출력
        CurseELECTRA_step += 1
        if CurseELECTRA_step % CurseELECTRA_eval_steps == 0:
            i = 0
            val_accuracy = 0

            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                CurseELECTRA_model.eval()        # 평가모드로 전환

                for val_batch in tqdm_auto(val_curse_loader, mininterval=0.01, leave=True):

                    # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
                    val_labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
                    val_outputs = CurseELECTRA_model(**inputs).logits     # 모형으로 결과 예측

                    loss = CurseELECTRA_criterion(val_outputs, val_labels)  
                    val_loss += loss

                    val_accuracy += accuracy_score(val_labels.cpu(), val_outputs.argmax(dim=1).cpu())
                    i += 1

                avg_val_loss = val_loss / len(val_curse_loader)

            val_accuracy /= i
            print('Step %d, validation loss: %.4f, accuracy_score: %.2f' % (CurseELECTRA_step, avg_val_loss, val_accuracy))
            
        avg_train_loss = train_loss / len(train_curse_loader)
        train_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
        n += 1

    train_accuracy /= n
    print('epoch %d, train loss: %.4f, accuracy_score: %.2f \n' % (epoch, avg_train_loss, train_accuracy))

  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

Step 546, validation loss: 0.1644, accuracy_score: 0.88
epoch 0, train loss: 0.3783, accuracy_score: 0.83 



  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

...

  0%|          | 0/182 [00:00<?, ?it/s]

Step 2184, validation loss: 0.0080, accuracy_score: 1.00
epoch 3, train loss: 0.0719, accuracy_score: 0.98 



  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

Step 2730, validation loss: 0.0029, accuracy_score: 1.00
epoch 4, train loss: 0.0486, accuracy_score: 0.99 



**KcELECTRA(챗봇 데이터 label 분류모델) 학습 진행**

In [42]:
# GPU 캐시 비우기 (GPU 메모리 확보)
torch.cuda.empty_cache()

In [51]:
for epoch in range(ChatELECTRA_epochs):
    n = 0
    train_accuracy = 0
    loss = 0
    train_loss = 0.0
    
    ChatELECTRA_model.train()
    for batch in tqdm_auto(train_chat_clf_loader, mininterval=0.01, leave=True):
        ChatELECTRA_optimizer.zero_grad()     # 그래디언트 초기화

        # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = ChatELECTRA_model(**inputs).logits    # 모형으로 결과 예측

        loss = ChatELECTRA_criterion(outputs, labels)
        train_loss += loss
        
        loss.backward()
        ChatELECTRA_optimizer.step()

        # eval_steps 마다 loss를 출력
        ChatELECTRA_step += 1
        if ChatELECTRA_step % ChatELECTRA_eval_steps == 0:
            i = 0
            val_accuracy = 0

            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                ChatELECTRA_model.eval()        # 평가모드로 전환

                for val_batch in tqdm_auto(val_chat_clf_loader, mininterval=0.01, leave=True):

                    # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
                    val_labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
                    val_outputs = ChatELECTRA_model(**inputs).logits     # 모형으로 결과 예측

                    loss = ChatELECTRA_criterion(val_outputs.argmax(dim=1).cpu(), val_labels.cpu())  
                    val_loss += loss

                    val_accuracy += accuracy_score(val_labels.cpu(), val_outputs.argmax(dim=1).cpu())
                    i += 1

                avg_val_loss = val_loss / len(val_curse_loader)

            val_accuracy /= i
            print('Step %d, validation loss: %.4f, accuracy_score: %.2f' % (CurseELECTRA_step, avg_val_loss, val_accuracy))
            
        avg_train_loss = train_loss / len(train_curse_loader)
        train_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
        n += 1

    train_accuracy /= n
    print('epoch %d, train loss: %.4f, accuracy_score: %.2f \n' % (epoch, avg_train_loss, train_accuracy))

  0%|          | 0/528 [00:00<?, ?it/s]

ValueError: ignored

**KoGPT2(챗봇 모델) 학습 진행**

In [ ]:
# GPU 캐시 비우기 (GPU 메모리 확보)
torch.cuda.empty_cache()

In [ ]:
for epoch in range(GPT2_epoch):
    loss = 0
    train_loss = 0.0
    
    GPT2_Model.train()
    for batch_idx, samples in enumerate(tqdm_auto(train_chat_loader, mininterval=0.01, leave=True)):
        GPT2_optimizer.zero_grad()       # optimizer 초기화(Gradient)

        # 모델 입력 텐서 GPU에 올리기
        token_ids, attention_mask, token_type_ids, label_ids = samples
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        label_ids = label_ids.to(device)

        out = GPT2_Model(
            input_ids=token_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=label_ids,
            )
        
        loss = out.loss
        loss.backward()
        GPT2_optimizer.step()
        
        train_loss += loss.item()

        # GPU 캐시 비우기 (GPU 메모리 확보)
        torch.cuda.empty_cache()
        
        # eval_steps 마다 loss를 출력
        GPT2_step += 1
        if GPT2_step % GPT2_eval_steps == 0:
            n = 0

            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                GPT2_Model.eval()

                for val_batch_idx, val_samples in enumerate(tqdm_auto(val_chat_loader, mininterval=0.01, leave=True)):
                    # 모델 입력 텐서 GPU에 올리기
                    val_token_ids, val_attention_mask, val_token_type_ids, val_label_ids = val_samples
                    val_token_ids = val_token_ids.to(device)
                    val_attention_mask = val_attention_mask.to(device)
                    val_token_type_ids = val_token_type_ids.to(device)
                    val_label_ids = val_label_ids.to(device)

                    val_out = GPT2_Model(
                            input_ids=val_token_ids,
                            attention_mask=val_attention_mask,
                            token_type_ids=val_token_type_ids,
                            labels=val_label_ids,
                            )
                    loss = out.loss

                    val_loss += loss.item()
                    torch.cuda.empty_cache()
                    
                val_loss /= val_batch_idx

            print('Step %d, validation loss: %.4f' % (GPT2_step, val_loss))
    
    # GPT2_scheduler.step()
    train_loss /= batch_idx
    print('epoch %d, train loss: %.4f \n' % (epoch, train_loss))

**테스트**

In [ ]:
import sys

In [ ]:
num_curse = 0
print("비속어 3회 입력(경고 3번)시 강제종료 됩니다.\n")

while 1:

    q = input("user > ").strip()
    # quit 입력시 챗봇 종료
    if q == "quit":
        break

    clean_text = clean(q)
    ELEC_input = ELECTRA_tokenizer.encode(clean_text, return_tensors="pt").to(device)

    with torch.no_grad():
        output = ELECTRA_model(ELEC_input).logits

        if output.argmax(dim=1).cpu() == 1:
            num_curse += 1
            if num_curse == 3:
                sys.exit("3회 경고 누적으로 강제종료 됩니다.")
            print("비속어가 감지 되었습니다. %d회 경고입니다.\n" % (num_curse))

        else:
            GPT2_input = GPT2_tokenizer.encode(clean_text, return_tensors="pt").to(device)
            gen_ids = GPT2_model.generate(
                GPT2_input,
                do_sample=True,
                max_length=30,
                top_p=0.5,
                top_k=5,
                repetition_penalty=1.0,
                no_repeat_ngram_size=2,
                temperature=0.5,
            )
    
            generated = GPT2_tokenizer.decode(gen_ids[0])
            generated = generated[generated.index("<sys>")+5 : generated.index("</s>")]
            
            print(f'Chatbot > {generated}')